In [1]:
# Import python directory
import sys
sys.path.append('..')

import torch
from PIL import Image
import matplotlib.pyplot as plt
from src.domain import Domain
from src.interpolation import SplineInter
from src.transformations import Affine2d
from src.plotting import view_image_2d, plot_grid_2d
from src.super_resolution_tools import *
from src.transformations import Affine2d
from src.LinearOperator import LinearOperator
from src.lanczos import lanczos_tridiag
# from src.conjugate_gradient import conjugate_gradient
from src.utils import * 
from src.optimization import *
import torch.func as func
from src.distance import SSDDistance
import time

torch.set_default_dtype(torch.float64)

In [2]:
As = torch.randn(10, 5,5)
As = As @ As.transpose(1,2)
As = As + 0.1 * torch.eye(5)
def A(x):
    # multiply A with x along 0th dimension without using einsum
    return torch.sum(As * (x**2).unsqueeze(1).unsqueeze(1), dim=0)


x = torch.randn(10)
Ax = A(x)
b = torch.sum(Ax,dim=1)
print(Ax)
print(b)



tensor([[105.7458,  19.2671,  21.9050, -34.2132,  13.2772],
        [ 19.2671,  87.8219, -24.7392,  -5.9891, -10.6498],
        [ 21.9050, -24.7392,  70.7085, -17.2125,  -8.6480],
        [-34.2132,  -5.9891, -17.2125,  83.5980,  -3.2949],
        [ 13.2772, -10.6498,  -8.6480,  -3.2949,  65.4531]])
tensor([125.9819,  65.7109,  42.0138,  22.8883,  56.1377])


In [70]:
def conjugate_gradient(A, b, x0, max_iter=50, tol=1e-15):
    x = x0.clone()
    r = b - A@x
    p = r.clone()
    rsold = r.dot(r)
        
    for i in range(max_iter):
        Ap = A@p
        alpha = rsold / p.dot(Ap)
        x = x + alpha * p  
        r = r - alpha * Ap  
        rsnew = r.dot(r)
        if torch.sqrt(rsnew/rsold) < tol:
            print(f"tol reached {i}")
            break
        p = r + (rsnew / rsold) * p
        rsold = rsnew
    # print(f"residual norm: {torch.sqrt(rsnew/r0)}")
    return x
    


def res_fn(x, cg_iter):
    Ax = A(x)
    n = Ax.shape[1]
    f0 = conjugate_gradient(Ax, b, torch.zeros(n), max_iter=cg_iter, tol=1e-15)
    
    return f0

def Jac_fn(wp_vec, cg_iter):
    return func.jacfwd(res_fn)(wp_vec, cg_iter).detach().squeeze()



In [71]:
cg_iter = 4
x0 = torch.randn(10)
res = res_fn(x0, cg_iter)
J = Jac_fn(x0, cg_iter)
dw = torch.randn_like(x0)  
dres = J @ dw

for i in range(40):
    h = 0.5**i
    rest = res_fn(x0 + h * dw, cg_iter)
    
    err0 = torch.norm(res - rest)
    err1 = torch.norm(res + h * dres - rest)
    print(f"Error at h={h:.3e}: {err0:.3e} vs {err1:.3e}")
    


Error at h=1.000e+00: 1.851e+00 vs 1.405e+00
Error at h=5.000e-01: 9.578e-01 vs 6.811e-01
Error at h=2.500e-01: 3.903e-01 vs 2.312e-01
Error at h=1.250e-01: 1.565e-01 vs 6.490e-02
Error at h=6.250e-02: 6.793e-02 vs 1.695e-02
Error at h=3.125e-02: 3.155e-02 vs 4.316e-03
Error at h=1.562e-02: 1.522e-02 vs 1.088e-03
Error at h=7.812e-03: 7.477e-03 vs 2.730e-04
Error at h=3.906e-03: 3.706e-03 vs 6.836e-05
Error at h=1.953e-03: 1.845e-03 vs 1.711e-05
Error at h=9.766e-04: 9.207e-04 vs 4.278e-06
Error at h=4.883e-04: 4.599e-04 vs 1.070e-06
Error at h=2.441e-04: 2.298e-04 vs 2.675e-07
Error at h=1.221e-04: 1.149e-04 vs 6.688e-08
Error at h=6.104e-05: 5.743e-05 vs 1.672e-08
Error at h=3.052e-05: 2.871e-05 vs 4.180e-09
Error at h=1.526e-05: 1.436e-05 vs 1.045e-09
Error at h=7.629e-06: 7.178e-06 vs 2.612e-10
Error at h=3.815e-06: 3.589e-06 vs 6.531e-11
Error at h=1.907e-06: 1.794e-06 vs 1.633e-11
Error at h=9.537e-07: 8.972e-07 vs 4.081e-12
Error at h=4.768e-07: 4.486e-07 vs 1.021e-12
Error at h